In [1]:
import os
import pathlib as Path
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random as rdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import datetime
from tensorflow.keras.applications import VGG19
from collections import deque
import random

In [2]:
def read_img(files):
    images = []
    labels = []
    for i in files:
        image = cv2.imread(i)
        image = cv2.resize(image,(150,150))
        images.append(image)
        labels.append(os.path.split(os.path.split(i)[0])[1])
    images = np.array(images)
    return images,labels

In [3]:
files = tf.io.gfile.glob('Mainpath3/Train/*/*.jpg')
rdm.shuffle(files)

In [4]:
train_data,train_labels = read_img(files)

In [5]:
val_files = tf.io.gfile.glob('Mainpath3/new_val/*/*.jpg')
rdm.shuffle(val_files)

In [6]:
val_data,val_labels = read_img(val_files)

In [7]:
test_files = tf.io.gfile.glob('Mainpath3/Test/*/*.jpg')
rdm.shuffle(val_files)


In [8]:
test_data,test_labels = read_img(test_files)

In [9]:
categories = {'Normal':0, 'Diabetes':1, 'Glaucoma':2, 'Cataract':3, 'Other':4,'Pathologocal_Myopia':5,'Macular_Deneration':6,'Hypertension':7}

In [10]:
train_mapped_lables = list(map(lambda x: categories.get(x, x), train_labels))
val_mapped_lables = list(map(lambda x: categories.get(x, x), val_labels))
test_mapped_lables = list(map(lambda x: categories.get(x, x), test_labels))

In [11]:
train_arr = np.array(train_mapped_lables)
val_arr = np.array(val_mapped_lables)
test_arr = np.array(test_mapped_lables)

In [12]:
print(train_data.shape)
print(train_arr.shape)

(6611, 150, 150, 3)
(6611,)


In [17]:
def create_cnn_model(input_shape, num_classes, filters=32, kernel_size=(3, 3), pool_size=(2, 2), dense_units=128, epochs=10, batch_size=32):
    model = Sequential()
    model.add(Conv2D(filters, kernel_size, activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size))
    model.add(Conv2D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling2D(pool_size))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
    
    
    input_dim = len([filters]) + len(flattened_kernel_size) + len(flattened_pool_size) + len([dense_units]) + 2  # +2 for epochs and batch_size
    model.add(Dense(24, input_dim=input_dim, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(num_classes, activation='linear'))
    model.compile(optimizer='adam', loss='mse')
    return model

In [18]:
class SimpleEnvironment:
    def __init__(self, hyperparameters):
        
        self.state_size = 8  
        self.action_size = 2
        self.current_step = 0
        self.max_steps = 100
        self.hyperparameters = hyperparameters

    def get_state_size(self):
        return self.state_size

    def get_action_size(self):
        return self.action_size

    def reset(self):
        
        kernel_pool_size = np.concatenate([np.array(hyperparameter_mapping['kernel_size']),
                                           np.array(hyperparameter_mapping['pool_size'])])
        
      
        state = np.array([hyperparameter_mapping['filters'], kernel_pool_size,
                          hyperparameter_mapping['dense_units'], hyperparameter_mapping['epochs'],
                          hyperparameter_mapping['batch_size']])
        
        return state.reshape(1, -1)  
    def step(self, action):
      
        self.current_step += 1
        current_hyperparameters = {
            'filters': np.random.choice(self.hyperparameters['filters']),
            'kernel_size': tuple(np.random.choice(self.hyperparameters['kernel_size'])),
            'pool_size': tuple(np.random.choice(self.hyperparameters['pool_size'])),
            'dense_units': np.random.choice(self.hyperparameters['dense_units']),
            'epochs': np.random.choice(self.hyperparameters['epochs']),
            'batch_size': np.random.choice(self.hyperparameters['batch_size'])
        }
        next_state = np.array([current_hyperparameters['filters'], current_hyperparameters['kernel_size'][0],
                              current_hyperparameters['kernel_size'][1], current_hyperparameters['pool_size'][0],
                              current_hyperparameters['pool_size'][1], current_hyperparameters['dense_units'],
                              current_hyperparameters['epochs'], current_hyperparameters['batch_size']])
        reward = 0.5  
        done = self.current_step >= self.max_steps
        return next_state, reward, done


In [19]:
# Deep Q-learning Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.batch_size = 32
        self.model = Sequential()
        self.model.add(Dense(24, input_dim=state_size, activation='relu'))
        self.model.add(Dense(24, activation='relu'))
        self.model.add(Dense(action_size, activation='linear'))
        self.model.compile(optimizer='adam', loss='mse')
        self.environment = None  # Initialize environment to None

    def set_up_environment(self, environment):
        self.environment = environment
        self.state_size = environment.get_state_size()
        self.action_size = environment.get_action_size()

    def set_environment(self, environment):
        self.environment = environment
        self.state = environment.reset()

    def select_action(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(self.action_size)
        q_values = self.model.predict(state.reshape(1, self.state_size))
        return np.argmax(q_values[0])

    def remember(self, state, action, reward, next_state):
        
        state = np.array(state).flatten()  
        next_state = np.array(next_state).flatten()  
        self.memory.append((state, action, reward, next_state))

    def replay(self):
        minibatch = np.array(self.memory, dtype=object)
        states = np.vstack(minibatch[:, 0])
        actions = minibatch[:, 1].astype(int)
        rewards = minibatch[:, 2]
        next_states = np.vstack(minibatch[:, 3])

        targets = rewards + self.gamma * np.max(self.model.predict(next_states), axis=1)
        targets_f = self.model.predict(states)
        targets_f[np.arange(len(actions)), actions] = targets

        self.model.fit(states, targets_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            self.epsilon = max(self.epsilon, self.epsilon_min)

    def plot_accuracy(self, accuracies):
        plt.plot(accuracies)
        plt.xlabel('Episode')
        plt.ylabel('Accuracy')
        plt.title('Accuracy per Episode')
        plt.show()

In [ ]:
hyperparameters = {
    'filters': [16, 32, 64],
    'kernel_size': [(3, 3), (4, 2), (3, 1), (2, 2)],
    'pool_size': [(2, 2), (2, 1), (3, 3), (3, 2)],
    'dense_units': [64, 128, 256],
    'epochs': [15,30,45],
    'batch_size': [16, 32, 64]
}


input_shape = (150, 150, 3)
num_classes = 5


agent = DQNAgent(8, len(hyperparameters['filters']) * len(hyperparameters['epochs']) * len(hyperparameters['batch_size']) * len(hyperparameters['dense_units']))

env = SimpleEnvironment(hyperparameters)


agent.set_up_environment(env)


termination_accuracy = 0.90
previous_accuracy = 0

episode_accuracies = []
total_reward = 0
for episode in range(10):  
    
    current_hyperparameters = {
        'filters': np.random.choice(hyperparameters['filters']),
        'kernel_size': tuple(random.choice(hyperparameters['kernel_size'])),
        'pool_size': tuple(random.choice(hyperparameters['pool_size'])),
        'dense_units': np.random.choice(hyperparameters['dense_units']),
        'epochs': np.random.choice(hyperparameters['epochs']),
        'batch_size': np.random.choice(hyperparameters['batch_size'])
    }

   
    model = create_cnn_model(input_shape, num_classes, **current_hyperparameters)




   
    print(f"\nTraining CNN with Hyperparameters: {current_hyperparameters}")
    model.fit(train_data, train_arr,epochs=current_hyperparameters['epochs'], batch_size=current_hyperparameters['batch_size'], validation_data=(val_data, val_arr))

    _, accuracy = model.evaluate(test_data, test_arr, verbose=0)

    episode_accuracies.append(accuracy)

 
    if accuracy > previous_accuracy or accuracy == 0:
        reward = 1.0
    elif accuracy == previous_accuracy:
        reward = 0.5
    else:
        reward = -1.0

    total_reward += reward

   
    current_state = np.array([current_hyperparameters['filters'], current_hyperparameters['kernel_size'][0],
                              current_hyperparameters['kernel_size'][1], current_hyperparameters['pool_size'][0],
                              current_hyperparameters['pool_size'][1], current_hyperparameters['dense_units'],
                              current_hyperparameters['epochs'], current_hyperparameters['batch_size']])
    next_state = current_state

    action = agent.select_action(current_state)
    agent.remember(current_state, action, reward, next_state)
    
    previous_accuracy = accuracy

   
    agent.replay()

   
    print(f"\nChanges made during Episode {episode + 1}:")
    print(f"Current Hyperparameters: {current_hyperparameters}")
    print(f"Model Accuracy: {accuracy}")
    print(f"Epsilon (Exploration): {agent.epsilon}\n")

   
    if accuracy >= termination_accuracy:
        print(f"Terminating training as accuracy reached {accuracy}")
        break


print(f"\nTotal Reward: {total_reward}")

agent.plot_accuracy(episode_accuracies)